In [1]:
# Dependencies
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///db.sqlite")

# Countries

Loads from a CSV copy of all [ISO country codes](https://www.iso.org/obp/ui/#search)

In [3]:
df = pd.read_csv("iso_countries.csv")

In [4]:
df.drop(columns=['French short name', 'Numeric'], inplace=True)

In [5]:
df.columns = ['short_name', 'code2', 'code3']

In [6]:
df

,short_name,code2,code3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND
...,...,...,...
244,Western Sahara*,EH,ESH
245,Yemen,YE,YEM
246,Zambia,ZM,ZMB
247,Zimbabwe,ZW,ZWE


In [7]:
df.to_sql('countries', engine, if_exists='replace')